In [2]:
import pandas as pd
import os
from abc import ABC
import string
from gensim.corpora import Dictionary
from torch.utils.data import Dataset
import gensim.downloader as api
from time import time
import gensim
import gensim.downloader as api
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from argparse import Namespace

import numpy as np
import json

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import pandas as pd
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm


def decodeList(x): return [w.strip("'") for w in x.split(' ')]


def pythonise(df):
    if 'is_fulltime' in df.keys():
        df.is_fulltime = df.is_fulltime.astype(str)
    if 'job_description' in df.keys():
        df['job_description'] = df['job_description'].apply(decodeList)
    if 'tfidf10' in df.keys():
        df['tfidf10'] = df['tfidf10'].apply(decodeList)

def load_data(name, send_computer_to_hell=True):
    nrows = None
    if not send_computer_to_hell:
        nrows = 1000
    df = pd.read_csv(f'./data/{name}.csv', nrows=nrows)
    train = pd.read_csv(f'./data/{name}-train.csv', nrows=nrows)
    test = pd.read_csv(f'./data/{name}-test.csv', nrows=nrows)
    val = pd.read_csv(f'./data/{name}-val.csv', nrows=nrows)
    # ugly af
    for d in [df, train, test, val]:
       pythonise(d)
    return df, train, test, val

In [50]:
df, train, test,val = load_data("task2", send_computer_to_hell=False)
df

,Unnamed: 0,job_description,category,tfidf10
0,0,"[have, you, had, years, experience, in, fresh,...",Retail & Consumer Products,"[the, sebastian, produce, and, for, store, wad..."
1,1,"[about, us, retail, safari, as, part, of, cpm,...",Retail & Consumer Products,"[safari, cpm, conversational, engaging, person..."
2,2,"[work, with, brand, you, love, wear, generous,...",Retail & Consumer Products,"[know, and, store, wear, sneakers, you, star, ..."
3,3,"[about, the, company, forever, new, is, one, o...",Retail & Consumer Products,"[late, new, to, beauty, her, forever, the, cel..."
4,4,"[sunglass, hut, is, global, retailer, with, st...",Retail & Consumer Products,"[chanel, ban, dolce, prada, oakley, gabbana, s..."
...,...,...,...,...
995,995,"[casual, pool, hourly, rate, inclusive, of, ca...",Government & Defence,"[swimming, casual, resuscitation, cardio, melv..."
996,996,"[manager, engineering, team, safework, nsw, de...",Government & Defence,"[of, engineering, safework, inspectors, the, n..."
997,997,"[aps, policy, officer, mandatory, baseline, se...",Government & Defence,"[officer, security, and, clearance, boarder, s..."
998,998,"[people, people, are, working, closely, with, ...",Government & Defence,"[changes, analyse, and, thinker, specification..."


## Utilities

### Vocab

In [4]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>", freqdist=None, tokens=-1):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}


        if freqdist is None:
            freqdist = {}
        self.freqdist = freqdist
        
        if tokens == -1:
            if len(self.freqdist) == 0:
                self.num_tokens = 0
            else:
                self.num_tokens = sum(self.freqdist.values())

        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
            
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token,
                "freqdist": self.freqdist}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    
    def add_token(self, token, count=1):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
            self.freqdist[token] = count
        self.num_tokens += 1
        return index
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    
    def lookup_frequency(self, token):
        return self.freqdist[token]/self.num_tokens
        
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

### Sequence Vocab

In [5]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

### Batch Generator

In [6]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
    ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

### Training Utils

In [7]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, encoding="utf8") as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

## Vectorisers

### Word2Vec

In [8]:
class Word2vecVectoriser(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, df, input_vocab, class_vocab, sent_embed, is_sequence, data_field, vector_len=-1, load_path=None):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
            rating_vocab (Vocabulary): maps class labels to integers
        """
        self.df = df
        self.data = df[data_field]
        self.input_vocab = input_vocab
        self.class_vocab = class_vocab
        self.is_sequence = is_sequence
        self.sent_embed = sent_embed 
        self.vector_len = vector_len
        if not load_path:
            self.model = self.model_base()
            self.build_vocab()
            self.train_model()
        else:
            self.model = gensim.models.KeyedVectors.load(load_path)
        
    def model_base(self):
        cores = 4
        model = Word2Vec(min_count=1,
                         window=2,
                         vector_size=100,
                         sample=6e-5, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         negative=20,
                         workers=cores-1)
        return model

    def build_vocab(self):
        t = time()
        self.model.build_vocab(self.data, progress_per=10)
        print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

    def train_model(self):
        t = time()
        self.model.train(self.data, total_examples=self.model.corpus_count, epochs=50, report_delay=1)
        print('Time to train : {} mins'.format(round((time() - t) / 60, 2)))
        
    def vectorize(self, words):
        """Create a collapsed one hot vector for the review
        Args:
            review (str): the review
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """

        indicies = [self.model.wv.get_index(word) for word in words]
        vectors = self.model.wv[indicies]

        if self.sent_embed:
            frequency = [self.input_vocab.lookup_frequency(word) for word in words]
            return sum([vec*freq for vec,freq in zip(vectors, frequency)])

        return vectors.flatten()
              
    @classmethod
    def from_dataframe(cls, df, is_sequence, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            review_df (pandas.DataFrame): the review dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the ReviewVectorizer
        """
        class_vocab = Vocabulary(add_unk=False)
        for category in sorted(set(df[feature_field])):
            class_vocab.add_token(category)

        word_counts = Counter()
        for title in df[data_field]:
            for token in title:
                if token not in string.punctuation:
                    word_counts[token] += 1

        if is_sequence:
            input_vocab = SequenceVocabulary()
        else:
            input_vocab = Vocabulary(add_unk=True)
        # Add top words if count > provided count
        for word, count in word_counts.items():
            if count > cutoff:
                input_vocab.add_token(word, count)

        return cls(input_vocab, class_vocab, sent_embed, is_sequence)

    @property
    def num_features(self):
        if self.sent_embed:
            return 100 #word2vec model size
        return self.vector_len * 100
    
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a ReviewVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the ReviewVectorizer class
        """
        review_vocab = Vocabulary.from_serializable(contents['input_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['class_vocab'])
        return cls(input_vocab=input_vocab, class_vocab=class_vocab)
    
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'input_vocab': self.input_vocab.to_serializable(),
                'class_vocab': self.class_vocab.to_serializable()}
    

In [9]:
def test_word2vec():
    df, train, test,val = load_data("task1", send_computer_to_hell=False)

    v = Word2vecVectoriser.from_dataframe(df)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]))

    v = Word2vecVectoriser.from_dataframe(df, data="job_description", cutoff=0)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]))
    

### One Hot

In [10]:
class OneHotVectoriser(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""

    def __init__(self, input_vocab, class_vocab, sent_embed, is_sequnce):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
            rating_vocab (Vocabulary): maps class labels to integers
            sent_embed (boolean): whether to use sentence embedding
        """
        self.input_vocab = input_vocab
        self.class_vocab = class_vocab
        self.sent_embed = sent_embed
        self.is_sequence = is_sequnce
        
    def vectorize(self, words, vector_length=-1):
        """Create a collapsed one hot vector for the review
        Args:
            review (str): the review
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        if self.is_sequence:
            indices = [self.input_vocab.begin_seq_index]
            indices.extend(self.input_vocab.lookup_token(token) for token in words)
            indices.append(self.input_vocab.end_seq_index)

            if self.sent_embed:
                if vector_length < 0:
                    vector_length = len(indices)

                out_vector = np.zeros(vector_length, dtype=np.int64)
                out_vector[:len(indices)] = indices
                out_vector[len(indices):] = self.input_vocab.mask_index
            else:
               out_vector = [] 
               for index in indices:
                   word_vector = np.zeros(len(self.input_vocab), dtype=np.int8)
                   word_vector[index] += 1
                   out_vector.append(word_vector)
        else:
            if self.sent_embed:
                out_vector = np.zeros(len(self.input_vocab), dtype=np.float32)
                for token in words:
                    out_vector[self.input_vocab.lookup_token(token)] += 1
            else:
               out_vector = [] 
               for token in words:
                   word_vector = np.zeros(len(self.input_vocab), dtype=np.int8)
                   word_vector[self.input_vocab.lookup_token(token)] += 1
                   out_vector.append(word_vector)

        return np.array(out_vector)

    @classmethod
    def from_dataframe(cls, df, is_sequence, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            review_df (pandas.DataFrame): the review dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the ReviewVectorizer
        """
        class_vocab = Vocabulary(add_unk=False)
        for category in sorted(set(df[feature_field])):
            class_vocab.add_token(category)

        word_counts = Counter()
        for title in df[data_field]:
            for token in title:
                if token not in string.punctuation:
                    word_counts[token] += 1

        if is_sequence:
            input_vocab = SequenceVocabulary()
        else:
            input_vocab = Vocabulary(add_unk=True)
        # Add top words if count > provided count
        for word, count in word_counts.items():
            if count > cutoff:
                input_vocab.add_token(word, count)

        return cls(input_vocab, class_vocab, sent_embed, is_sequence)

    @property
    def num_features(self):
        return len(self.input_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a ReviewVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the ReviewVectorizer class
        """
        is_sequence = contents['is_sequence']
        if is_sequence:
            input_vocab = SequenceVocabulary.from_serializable(contents['input_vocab'])
        else:
            input_vocab = Vocabulary.from_serializable(contents['input_vocab'])
        class_vocab = Vocabulary.from_serializable(contents['class_vocab'])
        return cls(input_vocab=input_vocab, class_vocab=class_vocab, is_sequence=is_sequence)

    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'input_vocab': self.input_vocab.to_serializable(),
                'class_vocab': self.class_vocab.to_serializable(),
                'is_sueqnce': self.is_sequence}

In [11]:
def test_onehot():
    df, train, test,val = load_data("task1", send_computer_to_hell=False)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=False, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=False, data_field="tfidf10", feature_field="is_fulltime", sent_embed=True)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)
    
    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="job_description", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="job_description", feature_field="is_fulltime", sent_embed=True)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)

# test_onehot()

### Pre-Trained

In [12]:
class PretrainedVectoriser(object):
    def __init__(self, sent_embed=False, path=None):
        self.vectoriser = self.create_pretrained(load_path=path)
        self.sent_embed = sent_embed
    
    def create_pretrained(self, load_path=None):
        if not load_path:
            pretrained_w2v = api.load('word2vec-google-news-300')
            return pretrained_w2v
        else:
            return gensim.models.KeyedVectors.load(load_path)
         
    def get_vectoriser(self):
        return self.vecotoriser

    def vectorise(self, words):
        indicies = [self.vecotoriser.get_index(word) for word in words]
        vectors = self.vectoriser[indicies]
        if self.sent_embed:
            return vectors.flatten()
        return vectors 

In [13]:
v = PretrainedVectoriser()  

In [14]:
v.vectoriser

## Dataset class

In [73]:
class JobDescriptionDataset(Dataset):
    def __init__(self, df, train, val, test, vectorizer, data_field="tfidf10", feature_field="is_fulltime"):
        """
        Args:
        review_df (pandas.DataFrame): the dataset
        vectorizer (Vectorizer): vectorizer instantiated from dataset
        """
        self.df = df
        self._vectorizer = vectorizer
        self.feature_field = feature_field
        self.data_field = data_field

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        def measure_len(context): return len(context)
        self._max_seq_length = max(map(measure_len, df[self.data_field])) + 2

        # if we don't have constant length 
        if not all(len(val) == self._max_seq_length for val in self.data_field):
            self.vector_len = self._max_seq_length
        else:
            self.vector_len = -1
        
        self.train = train
        self.val = val
        self.test = test
        self.train_size = len(self.train)
        self.validation_size = len(self.val)
        self.test_size = len(self.test)
        self._lookup_dict = {'train': (self.train, self.train_size),
                             'val': (self.val, self.validation_size),
                             'test': (self.test, self.test_size)}
        self.set_split('train')

        # Class weights
        class_counts = df[self.feature_field].value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.class_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / \
            torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, name, vectoriser=OneHotVectoriser, 
                                         data_field='tfidf10', 
                                         feature_field='is_fulltime',
                                         is_sequence=False,
                                         sent_embed=False,
                                         cutoff=25):
        """Load dataset and make a new vectorizer from scratch
        Args:
             name (str): name of task to load
        Returns:
            an instance of ReviewDataset
        """
        df, train, test, val = load_data(name)
        return cls(df, train, test, val, vectoriser.from_dataframe(df, is_sequence, data_field=data_field, 
            feature_field=feature_field, sent_embed=sent_embed, cutoff=cutoff), data_field=data_field, feature_field=feature_field)

    @classmethod
    def load_dataset_and_load_vectorizer(cls, name, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use

        Args:
            news_csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of NewsDataset
        """
        df, train, test, val = load_data(name)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, train, test, val, vectorizer.from_dataframe(df))

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file

        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of NewsVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return NewsVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json

        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        input_vector = \
            self._vectorizer.vectorize(row[self.data_field], vector_length=self.vector_len)
        class_index = \
            self._vectorizer.class_vocab.lookup_token(row[self.feature_field])
        return {'x_data': input_vector,
                'y_target': class_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size




### Setup

### actually train

## Convolution

In [16]:
class CNNVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""

    def __init__(self, input_vocab, class_vocab):
        self.input_vocab = input_vocab
        self.class_vocab = class_vocab

    def vectorize(self, title, vector_length=-1):
        """
        Args:
            title (list): the list of words separated by a space
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized title (numpy.array)
        """
        indices = [self.input_vocab.begin_seq_index]
        indices.extend(self.input_vocab.lookup_token(token)
                       for token in title)
        indices.append(self.input_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.input_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, news_df, data_field="job_description", feature_field='category', cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe

        Args:
            news_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary
        Returns:
            an instance of the NewsVectorizer
        """
        class_vocab = Vocabulary(add_unk=False)
        for category in sorted(set(news_df[feature_field])):
            class_vocab.add_token(category)
        
        word_counts = Counter()
        for title in news_df[data_field]:
            for token in title:
                if token not in string.punctuation:
                    word_counts[token] += 1

        input_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                input_vocab.add_token(word)

        return cls(input_vocab, class_vocab)

    @classmethod
    def from_serializable(cls, contents):
        input_vocab = \
            SequenceVocabulary.from_serializable(contents['input_vocab'])
        class_vocab =  \
            Vocabulary.from_serializable(contents['class_vocab'])

        return cls(input_vocab=input_vocab, class_vocab=class_vocab)

    def to_serializable(self):
        return {'input_vocab': self.input_vocab.to_serializable(),
                'class_vocab': self.class_vocab.to_serializable()}

# Binary Classification

## Data Loading

## Models

### Feed Forward Neural Network

In [17]:
class JobDescriptionNotSimpleClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(JobDescriptionNotSimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=int(num_features/2))
        self.fc2 = nn.Linear(in_features=int(num_features/2), out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        x_in = self.fc1(x_in)
        x_in = F.relu(x_in)
        y_out = self.fc2(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
            
        return y_out

In [18]:
class JobDescriptionSimpleClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(JobDescriptionSimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

#### Training

In [21]:
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='task1-model.pth',
    name='task1',
    save_dir='./model_storage/task1/',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.01,
    num_epochs=20,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
)

In [22]:
def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

# dataset and vectorizer
# dataset2 = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.input_name, Word2vecVectoriser)
# vectorizer2 = dataset2.get_vectorizer()
dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, data_field='tfidf10', feature_field='is_fulltime', 
    is_sequence=False, sent_embed=True)
vectorizer = dataset.get_vectorizer()
# model
# classifier = JobDescriptionClassifier(num_features=len(vectorizer.input_vocab))
classifier = JobDescriptionNotSimpleClassifier(num_features=vectorizer.num_features)
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

#### Training Loop

In [23]:
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in tqdm(enumerate(batch_generator), colour='white'):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

#### Evaluation

In [24]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [25]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.461
Test Accuracy: 90.87


### CNN Conv1D Model

In [65]:
class CNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(CNNClassifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
        
            
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=1),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2),
            nn.ELU()
        )

        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        # embed and permute so features are channels
        x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_embedded)

        # average and remove the extra dimension
        remaining_size = features.size(dim=2)
        features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

    


#### Train Setup

In [66]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [67]:
args = Namespace(
    # Data and Path hyper parameters
    name="task1",
    news_csv="./data/ag_news/news_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/document_classification",
    # Model hyper parameters
    glove_filepath='/home/dadams/Files/uni/nlp/data/glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=20, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    # Dataset params
    data_field='tfidf10', 
    feature_field='is_fulltime', 
    is_sequence=True, 
    sent_embed=True
)

In [41]:


def do_train(args):
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir,
                                             args.model_state_file)

        print("Expanded filepaths: ")
        print("\t{}".format(args.vectorizer_file))
        print("\t{}".format(args.model_state_file))

    # Check CUDA
    if not torch.cuda.is_available():
        args.cuda = False

    args.device = torch.device("cuda" if args.cuda else "cpu")
    print("Using CUDA: {}".format(args.cuda))

    # Set seed for reproducibility
    set_seed_everywhere(args.seed, args.cuda)

    # handle dirs
    handle_dirs(args.save_dir)
    # args.use_glove = True

    if args.reload_from_files:
        # training from a checkpoint
        dataset = NewsDataset.load_dataset_and_load_vectorizer(args.news_csv,
                                                               args.vectorizer_file)
    else:
        dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, data_field=args.data_field, feature_field=args.feature_field, 
            is_sequence=args.is_sequence, sent_embed=args.sent_embed)
        # create dataset and vectorizer
        # dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, vectoriser=CNNVectorizer)
        # dataset.save_vectorizer(args.vectorizer_file)
    vectorizer = dataset.get_vectorizer()

    # Use GloVe or randomly initialized embeddings
    if args.use_glove:
        words = vectorizer.input_vocab._token_to_idx.keys()
        embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                           words=words)
        print("Using pre-trained embeddings")
    else:
        print("Not using pre-trained embeddings")
        embeddings = None


    print(len(vectorizer.class_vocab))

    classifier = CNNClassifier(embedding_size=args.embedding_size, 
                                num_embeddings=len(vectorizer.input_vocab),
                                num_channels=args.num_channels,
                                hidden_dim=args.hidden_dim, 
                                num_classes=len(vectorizer.class_vocab), 
                                dropout_p=args.dropout_p,
                                pretrained_embeddings=embeddings,
                                padding_idx=0)
    
    classifier = classifier.to(args.device)
    dataset.class_weights = dataset.class_weights.to(args.device)

    loss_func = nn.CrossEntropyLoss(dataset.class_weights)
    optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                               mode='min', factor=0.5,
                                               patience=1)

    train_state = make_train_state(args)

    epoch_bar = tqdm(desc='training routine', 
                              total=args.num_epochs,
                              position=0)

    dataset.set_split('train')
    train_bar = tqdm(desc='split=train',
                              total=dataset.get_num_batches(args.batch_size), 
                              position=1, 
                              leave=True)
    dataset.set_split('val')
    val_bar = tqdm(desc='split=val',
                            total=dataset.get_num_batches(args.batch_size), 
                            position=1, 
                            leave=True)


    for epoch_index in range(args.num_epochs):
    # for epoch_index in range(2):
        train_state['epoch_index'] = epoch_index
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()
    #         print(batch_dict['x_data'])
            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
    # compute the loss & accuracy on the test set using the best available model

    classifier.load_state_dict(torch.load(train_state['model_filename']))

    classifier = classifier.to(args.device)
    dataset.class_weights = dataset.class_weights.to(args.device)
    loss_func = nn.CrossEntropyLoss(dataset.class_weights)

    dataset.set_split('test')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # compute the output
        y_pred =  classifier(batch_dict['x_data'])

        # compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

    train_state['test_loss'] = running_loss
    train_state['test_acc'] = running_acc

    print("Test loss: {};".format(train_state['test_loss']))
    print("Test Accuracy: {}".format(train_state['test_acc']))



In [42]:
do_train(args)

Expanded filepaths: 
	model_storage/document_classification/vectorizer.json
	model_storage/document_classification/model.pth
Using CUDA: True
Not using pre-trained embeddings
2


training routine:   0%|          | 0/20 [00:00<?, ?it/s]

split=train:   0%|          | 0/96 [00:00<?, ?it/s]

split=val:   0%|          | 0/27 [00:00<?, ?it/s]

Test loss: 0.6107524908505954;
Test Accuracy: 84.07451923076924


#### Setup 

In [83]:
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)
# args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = NewsDataset.load_dataset_and_load_vectorizer(args.news_csv,
                                                           args.vectorizer_file)
else:
    dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, data_field=args.data_field, feature_field=args.feature_field, 
        is_sequence=args.is_sequence, sent_embed=args.sent_embed)
    # create dataset and vectorizer
    # dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, vectoriser=CNNVectorizer)
    # dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.input_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None


print(len(vectorizer.class_vocab))

classifier = CNNClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.input_vocab),
                            num_channels=args.num_channels,
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(vectorizer.class_vocab), 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)

Expanded filepaths: 
	model_storage/document_classification/vectorizer.json
	model_storage/document_classification/model.pth
Using CUDA: True
Not using pre-trained embeddings
2


#### Train

In [84]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)


for epoch_index in range(args.num_epochs):
# for epoch_index in range(2):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset

    # setup: batch generator, set loss and acc to 0, set train mode on

    dataset.set_split('train')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()

    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is these 5 steps:

        # --------------------------------------
        # step 1. zero the gradients
        optimizer.zero_grad()
#         print(batch_dict['x_data'])
        # step 2. compute the output
        y_pred = classifier(batch_dict['x_data'])
        
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # step 4. use loss to produce gradients
        loss.backward()

        # step 5. use optimizer to take gradient step
        optimizer.step()
        # -----------------------------------------
        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

        # update bar
        train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                              epoch=epoch_index)
        train_bar.update()

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset

    # setup: batch generator, set loss and acc to 0; set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):

        # compute the output
        y_pred =  classifier(batch_dict['x_data'])

        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)
        val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                        epoch=epoch_index)
        val_bar.update()

    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

    train_state = update_train_state(args=args, model=classifier,
                                     train_state=train_state)

    scheduler.step(train_state['val_loss'][-1])

    if train_state['stop_early']:
        break

    train_bar.n = 0
    val_bar.n = 0
    epoch_bar.update()
    


training routine:   0%|          | 0/20 [00:00<?, ?it/s]

split=train:   0%|          | 0/96 [00:00<?, ?it/s]

split=val:   0%|          | 0/27 [00:00<?, ?it/s]

#### Evaluate

In [85]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [86]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

Test loss: 0.41238822501439315;
Test Accuracy: 92.00721153846155


### CNN Conv1D (full job description)

In [87]:
args = Namespace(
    # Data and Path hyper parameters
    name="task1",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/document_classification",
    # Model hyper parameters
    glove_filepath='/home/dadams/Files/uni/nlp/data/glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=20, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    # Dataset params
    data_field='job_description', 
    feature_field='is_fulltime', 
    is_sequence=True, 
    sent_embed=True
) 

In [88]:
do_train(args)

Expanded filepaths: 
	model_storage/document_classification/vectorizer.json
	model_storage/document_classification/model.pth
Using CUDA: True
Not using pre-trained embeddings
2


training routine:   0%|          | 0/20 [00:00<?, ?it/s]

split=train:   0%|          | 0/96 [00:00<?, ?it/s]

split=val:   0%|          | 0/27 [00:00<?, ?it/s]

Test loss: 0.37806626351980066;
Test Accuracy: 92.3076923076923


## Comparison / Discussion

# Multi-class Classification

## Models

### Vanilla RNN

In [111]:
class ElmanRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        """
            input_size (int): Size of the input vectors 
                i.e. dimension of the embeddings
            hidden_size (int): Size of the hidden state vectors
            batch_first (bool): whether the 0th dimension is batch
        """
        super(ElmanRNN, self).__init__()
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.batch_first = batch_first
        self.hidden_size = hidden_size
        
    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))
    
    def forward(self, x_in, initial_hidden=None):
        """Forward pass of ElmanRNN
        
        Args:
            x_in (torch.Tensor) an input data tensor
                If self.batch_first: x_in.shape = (batch, seq_size, feat_size)
                Else: x_in.shape = (seq_size, batch, feat_size)
            initial_hidden (torch.Tensor): initial hiden state for the RNN
        Returns:
            hiddens (torch.Tensor): The outputs of the RNN at each time step.
                If self.batch_first: hiddens.shape = (batch, seq_size, hidden_size)
                Else: hiddens.shape = (seq_size, batch, hidden_size)
        """
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
        else:
            seq_size, batch_size, feat_size = x_in.size()
        
        hiddens = []
        
        if initial_hidden_hidden is None:
            initial_hidden = self._initial_hidden(batch_size)
            initial_hidden = initial_hidden.to(x_in.device)
            
        hidden_t = initial_hidden
        hiddens = []
        for t in range(seq_size):
            hidden_t = self.rnn_cell(x_in[t], hidden_t)
            hiddens.append(hidden_t) # doesn't include the initial hidden state
        
        hiddens = torch.stack(hiddens)
        
        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)
            
        return hiddens
    
class RNNCategoryClasifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings,
                 num_classes, rnn_hidden_size, 
                 batch_first=True, padding_idx=0):
        """
        Args:
            embedding_size (int): The size of the word embeddings
            num_embeddings (int): number of words to embed (vocab size)
            num_classes (int): Size of the prediction vector (number of categories)
            rnn_hidden_size (int): Size of the RNN's hidden state
            batch_first (bool): Informs whether the input tensors will have
                batch first or the sequence on the 0th dimension
            padding_idx (int): The index for the tensor padding:
                see torch.nn.Embedding
        """
        self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                embedding_dim=embedding_size,
                                padding_idx=padding_idx)
        self.rnn = ElmanRNN(input_size=embedding_size, 
                            hidden_size=rnn_hidden_size,
                            batch_first=batch_first)
        self.fc1 = nn.Linear(in_features=rnn_hidden_size,
                            out_features=rnn_hidden_size)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size,
                            out_features=num_classes)
        
    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, input_dim)
            x_lengths (torch.Tensor): the lengths of each sequence in the batch.
                They are used to find the final vector of each sequence
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, output_dim)
        """
        
        x_embedded = self.vectorizer.vectorize(x_in)
        y_out = self.rnn(x_embedded)
        
        if x_lengths is not None:
            y_out = column_gather(y_out, x_lengths)
        else:
            y_out = y_out[:, -1, :]
        
        y_out = F.relu(self.fc1(F.dropout(y_out, 0.5)))
        y_out = self.fc2(F.dropout(y_out, 0.5))
        
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
            
        return y_out

In [89]:
args = Namespace(
    input_name='task2',
)

In [109]:
df, train, test, val = load_data("task2", send_computer_to_hell=False)
v = Word2vecVectoriser.from_dataframe(df, data="job_description", feature_field='category', cutoff=25)


TypeError: from_dataframe() got an unexpected keyword argument 'feature_field'

In [103]:
vocab = vectorizer.input_vocab
all_vectors = [vectorizer.vectorize(vocab.lookup_index(i)) for i in range(vocab.num_tokens - 1)]
#RNNCategoryClasifier(vectorizer, 500, len(vectorizer.class_vocab), 20)
all_vectors


vectorizer.vectorize(['hello', 'word'])

KeyError: 'is_fulltime'

### LSTM

## Comparison / Discussion